In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **STEP 1: Clone & Build Darknet**


In [ ]:
import os

# Check if darknet directory exists
if not os.path.exists('darknet'):
    # Clone Darknet if it doesn't exist
    !git clone https://github.com/AlexeyAB/darknet.git
    %cd darknet
else:
    # If it exists, just change directory to darknet
    %cd darknet

# Compile for GPU (disable CUDNN_HALF for compatibility)
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=1/CUDNN_HALF=0/' Makefile
!make

Cloning into 'darknet'...
remote: Enumerating objects: 15909, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 15909 (delta 2), reused 4 (delta 1), pack-reused 15900 (from 1)
Receiving objects: 100% (15909/15909), 14.43 MiB | 16.99 MiB/s, done.
Resolving deltas: 100% (10710/10710), done.
/content/darknet
mkdir -p ./obj/
mkdir -p backup
mkdir -p results
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: 

## **STEP 2: Dataset Setup**
Ensure dataset is structured as follows inside Google Drive:
```
classes=1
train=/path/to/train.txt
valid=/path/to/test.txt
names=/path/to/obj.names
backup=backup/
```

In [ ]:
import os

# Paths

image_dir = "/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/images/train"  # CAMO dataset images
label_dir = "/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/labels/train"  # CAMO dataset labels
train_txt = "/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/train.txt"  # Output train.txt


# Ensure directories exist
os.makedirs(image_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)
os.makedirs(os.path.dirname(train_txt), exist_ok=True)

# Get image and label files
image_extensions = (".jpg", ".png")
images = [f for f in os.listdir(image_dir) if f.lower().endswith(image_extensions)]
labels = [f for f in os.listdir(label_dir) if f.lower().endswith(".txt")]

# Match images with labels
valid_images = []
for img in images:
    label_file = os.path.splitext(img)[0] + ".txt"
    if label_file in labels:
        valid_images.append(os.path.join(image_dir, img))
    else:
        print(f"Missing label for {img}")

# Check for labels without images
for lbl in labels:
    img_file = os.path.splitext(lbl)[0] + ".jpg"  # Adjust extension if needed
    if img_file not in images and os.path.splitext(lbl)[0] + ".png" not in images:
        print(f"Missing image for {lbl}")

# Update train.txt with valid image paths
with open(train_txt, "w") as f:
    for img_path in valid_images:
        f.write(f"{img_path}\n")

print(f"Updated {train_txt} with {len(valid_images)} valid image-label pairs")

Updated /content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/train.txt with 1000 valid image-label pairs


In [ ]:
# Generate train.txt, test.txt, valid.txt
import os
import glob
from sklearn.model_selection import train_test_split

# Define paths
base_image_dir = '/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/images'
train_image_dir = os.path.join(base_image_dir, 'train')
val_image_dir = os.path.join(base_image_dir, 'val')
output_dir = '/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset'
train_txt_hjei5qreljpq = "/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/train.txt" # Path to train.txt from cell HJEi5QRELjpq


# Read valid image paths from train.txt generated by HJEi5QRELjpq
try:
    with open(train_txt_hjei5qreljpq, 'r') as f:
        train_files = [line.strip() for line in f if line.strip()]
except FileNotFoundError:
    print(f"Error: {train_txt_hjei5qreljpq} not found. Please run cell HJEi5QRELjpq first.")
    train_files = [] # Initialize as empty to prevent further errors


if not train_files:
    print("No valid image paths found in the train.txt generated by HJEi5QRELjpq. Please check the previous cell's output.")
else:
    # Collect all image files from train and val directories
    all_image_files = []
    all_image_files.extend(glob.glob(os.path.join(train_image_dir, '*.jpg')))
    all_image_files.extend(glob.glob(os.path.join(val_image_dir, '*.jpg')))
    all_image_files.extend(glob.glob(os.path.join(train_image_dir, '*.png'))) # Also include png
    all_image_files.extend(glob.glob(os.path.join(val_image_dir, '*.png'))) # Also include png


    # Ensure all_image_files contains the training files for splitting
    # This step might not be strictly necessary if train_files are already included in glob results,
    # but it ensures robustness in case of path variations.
    all_image_files_set = set(all_image_files)
    missing_train_files_in_all = [f for f in train_files if f not in all_image_files_set]
    if missing_train_files_in_all:
        print(f"Warning: Some training files from {train_txt_hjei5qreljpq} were not found in the specified image directories:")
        for mf in missing_train_files_in_all:
            print(mf)


    # Exclude training files from the list before splitting for test and validation
    test_val_files = [f for f in all_image_files if f not in set(train_files)]


    if not test_val_files:
         print("No image files found outside of the training set for test/validation split.")
         test_files = []
         valid_files = []
    else:
        # Split the remaining images into test and validation sets
        test_files, valid_files = train_test_split(test_val_files, test_size=0.5, random_state=42)


    # Create the directory for the output files if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Write the train.txt with the valid image paths from HJEi5QRELjpq
    with open(os.path.join(output_dir, 'train.txt'), 'w') as f:
        f.write('\n'.join(train_files))
    print(f"Successfully generated train.txt in {output_dir} with {len(train_files)} images.")

    # Write the test.txt and valid.txt
    with open(os.path.join(output_dir, 'test.txt'), 'w') as f:
        f.write('\n'.join(test_files))
    print(f"Successfully generated test.txt in {output_dir} with {len(test_files)} images.")

    with open(os.path.join(output_dir, 'valid.txt'), 'w') as f:
        f.write('\n'.join(valid_files))
    print(f"Successfully generated valid.txt in {output_dir} with {len(valid_files)} images.")

Successfully generated train.txt in /content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset with 1000 images.
Successfully generated test.txt in /content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset with 125 images.
Successfully generated valid.txt in /content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset with 125 images.


In [ ]:
# Create obj.names
with open('/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset/obj.names', 'w') as f:
    f.write('camouflaged_object\n')

In [ ]:
# Create obj.data
with open('/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/obj.data', 'w') as f:
    f.write('classes = 1\n')
    f.write('train = /content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset/train.txt\n')
    f.write('valid = /content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset/valid.txt\n')
    f.write('names = /content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset/obj.names\n')
    f.write('backup = /content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset/backup/\n')

In [ ]:
# Create backup directory
!mkdir /content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset/backup

mkdir: cannot create directory ‘/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset/backup’: File exists


## **STEP 3: Configure YOLOv4-Tiny**
Copy default config and edit for CAMO dataset (1 class).

In [ ]:
!cp cfg/yolov4-tiny.cfg cfg/yolov4-tiny-custom.cfg

# TODO: Edit yolov4-tiny-custom.cfg manually in Colab or upload ready-made version:
# - batch=64, subdivisions=16
# - max_batches=4000, steps=3200,3600
# - classes=1
# - filters=18 (before each [yolo] layer)

In [ ]:
# ===========================================================================================
# STEP next : Edit the yolov4-tiny-custom.cfg file to change the number of classes and filters
# ===========================================================================================

cfg_path = "/content/darknet/cfg/yolov4-tiny-custom.cfg"

with open(cfg_path, 'r') as f:
    cfg_content = f.readlines()

with open(cfg_path, 'w') as f:
    for line in cfg_content:
        # Use strip() to remove leading/trailing whitespace before checking
        stripped_line = line.strip()
        if stripped_line.startswith('classes='):
            f.write('classes=1\n')
        elif stripped_line.startswith('filters='):
            # Filters need to be calculated as (classes + 5) * 3
            f.write('filters=18\n') # (1 + 5) * 3 = 18
        elif stripped_line.startswith('max_batches='):
            f.write('max_batches=4000\n')
        elif stripped_line.startswith('steps='):
            f.write('steps=3200,3600\n')
        else:
            f.write(line)

print(f"Updated {cfg_path} with classes=1, corresponding filters=18, max_batches=6000, and steps=4800,5400.")

Updated /content/darknet/cfg/yolov4-tiny-custom.cfg with classes=1, corresponding filters=18, max_batches=6000, and steps=4800,5400.


## **STEP 4: Download Pretrained Weights**

In [ ]:

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29


--2025-08-29 20:15:16--  https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/75388965/28807d00-3ea4-11eb-97b5-4c846ecd1d05?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-08-29T20%3A59%3A10Z&rscd=attachment%3B+filename%3Dyolov4-tiny.conv.29&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-08-29T19%3A58%3A36Z&ske=2025-08-29T20%3A59%3A10Z&sks=b&skv=2018-11-09&sig=%2F%2BPJMWcqlMsre72R9j2c9sx3iJspmu%2B8e%2B3Uw2lwyQQ%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1NjQ5ODgxNiwibmJmIjoxNzU2NDk4NTE2LCJwYXRoIjoicmVsZWFzZ

## **STEP 5: Train YOLOv4-Tiny**

In [ ]:
%cd /content/darknet
!./darknet detector train   "/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/obj.data"   "/content/darknet/cfg/yolov4-tiny-custom.cfg"   "yolov4-tiny.conv.29"   -dont_show -map

/content/darknet
./darknet: error while loading shared libraries: libcuda.so.1: cannot open shared object file: No such file or directory


## **STEP 6: Run Inference**

In [ ]:
%cd /content/darknet
!./darknet detector test   "/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/obj.data"   "/content/darknet/cfg/yolov4-tiny-custom.cfg"   "/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset/backup/yolov4-tiny-custom_final.weights"   -dont_show   "/content/drive/MyDrive/Colab Notebooks/Dataset/CAMO/images/test/0001.jpg"

from IPython.display import Image
Image(filename='predictions.jpg')

/content/darknet
./darknet: error while loading shared libraries: libcuda.so.1: cannot open shared object file: No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'predictions.jpg'

## **STEP 7: Evaluate on Test Set**

In [ ]:
%cd /content/darknet
!./darknet detector map   "/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/obj.data"   "/content/darknet/cfg/yolov4-tiny-custom.cfg"   "/content/drive/MyDrive/Colab_Notebooks/Dataset/CAMO/CAMO_Dataset/backup/yolov4-tiny-custom_best.weights"